In [1]:
import xarray as xr
import numpy as np
import rioxarray
from rasterio.enums import Resampling
import glob
import pandas as pd
import time
from dask.diagnostics import ProgressBar
import xesmf as xe
import dask.distributed

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"

in_dir = dir + ("data/processed_data/landsat/mosaic/")
out_dir = dir + ("data/processed_data/landsat/resampled/")

target_image = xr.open_dataset(
    dir +
    "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max_1984_2013.nc"
)["LAI"].isel(time=5)
arr = xr.open_dataarray("../data/arr_id.nc")
target_image = target_image.rio.write_crs("EPSG:4326").rename({
    "latitude": "lat",
    "longitude": "lon"
})
# Grid of the target image 
lat1 = target_image["lat"].values
lon1 = target_image["lon"].values

Resmpling TM and ETM NDVI and make the values between the two sensors consistent

In [ ]:
for i in np.arange(1995,2015):
    print("resampling NDVI TM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_NDVI_"+str(i)+"_TM5.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_reproj = ref_image.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_TM_v2.nc")

In [ ]:
for i in np.arange(1999,2015):
    print("Correcting and resampling ETM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_NDVI_"+str(i)+"_ETM.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_image = ref_image/10000
    ref_image_cor = (ref_image+0.015)/1.095
    ref_image_cor = ref_image_cor*10000
    ref_reproj = ref_image_cor.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v2.nc")

In [ ]:
# Correcting for large values outside of the region. 
for i in range(1999,2015):
    print(i)
    da=xr.open_dataset(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v2.nc")["band_data"]
    da = da.where(da<10000)
    da.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v3.nc")

fnames= []
for i in np.arange(1984,2015):
    if i<=1999:
        fnames.append(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_TM_v2.nc")
        continue
    fnames.append(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v3.nc") 
t=pd.date_range(start="1984",end="2015",freq="A").year
ds = xr.open_mfdataset(fnames,concat_dim="time",combine="nested").rename_vars({"band_data":"NDVI"})
ds["time"]=t
da = ds["NDVI"]
da.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_v2.nc")

Resmpling TM and ETM NIR and make the values between the two sensors consistent

In [ ]:
for i in np.arange(1984,1995):
    print("resampling NIR TM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_NIR_"+str(i)+"_TM5.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_reproj = ref_image.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/NIR_resampled_"+str(i)+"_TM_v2.nc")

In [ ]:
for i in np.arange(1999,2015):
    print("Correcting and resampling NIR ETM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_NIR_"+str(i)+"_ETM.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_image = ref_image/10000
    ref_image_cor = (ref_image+0.004)/1.052
    ref_image_cor = ref_image_cor*10000
    ref_reproj = ref_image_cor.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/NIR_resampled_"+str(i)+"_ETMCor_v2.nc")

In [ ]:
# Correcting for large values outside of the region. 
for i in range(1999,2015):
    print(i)
    da=xr.open_dataset(dir+"data/processed_data/landsat/resampled/NIR_resampled_"+str(i)+"_ETMCor_v2.nc")["band_data"]
    da = da.where(da<10000)
    da.to_netcdf(dir+"data/processed_data/landsat/resampled/NIR_resampled_"+str(i)+"_ETMCor_v3.nc")
fnames= []
for i in np.arange(1984,2015):
    if i<=1999:
        fnames.append(dir+"data/processed_data/landsat/resampled/NIR_resampled_"+str(i)+"_TM_v2.nc")
        continue
    fnames.append(dir+"data/processed_data/landsat/resampled/NIR_resampled_"+str(i)+"_ETMCor_v3.nc") 
t=pd.date_range(start="1984",end="2015",freq="A").year
ds = xr.open_mfdataset(fnames,concat_dim="time",combine="nested").rename_vars({"band_data":"NIR"})
ds["time"]=t
da = ds["NIR"]
da.to_netcdf(dir+"data/processed_data/landsat/resampled/NIR_resampled_v2.nc")

In [ ]:
for i in np.arange(1984,2013):
    print("resampling SWIR TM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_SWIR2_"+str(i)+"_TM5.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_reproj = ref_image.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/SWIR2_resampled_"+str(i)+"_TM_v2.nc")

In [ ]:
for i in np.arange(1999,2015):

    print("Correcting and resampling SWIR ETM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_SWIR2_"+str(i)+"_ETM.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_image = ref_image/10000
    ref_image_cor = (ref_image-0.003)/0.903
    ref_image_cor = ref_image_cor*10000
    ref_reproj = ref_image_cor.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/SWIR2_resampled_"+str(i)+"_ETMCor_v2.nc")

In [2]:
for i in np.arange(1984,2013):
    print("Calculate NBR and NIRv TM: "+str(i))
    nir = xr.open_dataarray(in_dir + "mosaic_NIR_"+str(i)+"_TM5.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    swir = xr.open_dataarray(in_dir + "mosaic_SWIR2_"+str(i)+"_TM5.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ndvi = xr.open_dataarray(in_dir + "mosaic_NDVI_"+str(i)+"_TM5.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    nbr = (nir-swir)/(nir+swir)
    nirv = ndvi*nir
    nbr_reproj = nbr.rio.reproject(4326,esampling=Resampling.nearest)
    nirv_reproj = nirv.rio.reproject(4326,esampling=Resampling.nearest)
    nbr_reproj = nbr_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    nirv_reproj = nirv_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    nbr_regridded = nbr_reproj.interp(lat=lat1,lon=lon1,method='linear')
    nirv_regridded = nirv_reproj.interp(lat=lat1,lon=lon1,method='linear')
    nbr_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/nbr_resampled_"+str(i)+"_TM_v2.nc")
    nirv_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/nirv_resampled_"+str(i)+"_TM_v2.nc")


Calculate NBR and NIRv TM: 1984


/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/scipy/interpolate/_interpolate.py:698: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/scipy/interpolate/_interpolate.py:701: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Calculate NBR and NIRv TM: 1985


/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/scipy/interpolate/_interpolate.py:701: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Calculate NBR and NIRv TM: 1986


...

/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/scipy/interpolate/_interpolate.py:698: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/scipy/interpolate/_interpolate.py:701: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Calculate NBR and NIRv TM: 1989


/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/scipy/interpolate/_interpolate.py:698: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/scipy/interpolate/_interpolate.py:701: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Calculate NBR and NIRv TM: 1990


/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [ ]:
for i in np.arange(1999,2015):
    print("Calculate NBR and NIRv ETM: "+str(i))
    swir = xr.open_dataarray(in_dir + "mosaic_SWIR2_"+str(i)+"_ETM.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    swir = swir/10000
    swir_cor = (swir-0.003)/0.903
    swir_cor = swir_cor*10000
    nir = xr.open_dataarray(in_dir + "mosaic_NIR_"+str(i)+"_ETM.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    nir = nir/10000
    nir_cor = (nir+0.004)/1.052
    nir_cor = nir_cor*10000
    ndvi = xr.open_dataarray(in_dir + "mosaic_NDVI_"+str(i)+"_ETM.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ndvi = ndvi/10000
    ndvi_cor = (ndvi+0.015)/1.095
    ndvi_cor = ndvi_cor*10000
 
    nbr_cor = (nir_cor-swir_cor)/(nir_cor+swir_cor)
    nirv_cor = ndvi_cor*nir_cor
    nbr_cor_reproj = nbr_cor.rio.reproject(4326,esampling=Resampling.nearest)
    nirv_cor_reproj = nirv_cor.rio.reproject(4326,esampling=Resampling.nearest)
    nbr_cor_reproj = nbr_cor_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    nirv_cor_reproj = nirv_cor_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    nbr_cor_regridded = nbr_cor_reproj.interp(lat=lat1,lon=lon1,method='linear')
    nirv_cor_regridded = nirv_cor_reproj.interp(lat=lat1,lon=lon1,method='linear')
    nbr_cor_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/nbr_resampled_"+str(i)+"_ETM_v2.nc")
    nirv_cor_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/nirv_resampled_"+str(i)+"_ETM_v2.nc")
